In [5]:
# import library for displaying images after curl 
import cv2
import numpy as np
import requests
import os
import pandas as pd


In [6]:
DATASET_FOLDER = '../dataset/'
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

In [ ]:
image_links = sample_test['image_link']
image_links = image_links.tolist()

# download the images using curl
for i in range(len(image_links)):
    img_data = requests.get(image_links[i]).content
    with open('../images_curl/image_'+str(i)+'.jpg', 'wb') as handler:
        handler.write(img_data)

In [8]:
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))

In [16]:
print(np.unique(train['image_link']).shape[0]/train.shape[0])
# print the difference in number of unique vs total images
print(train.shape[0] - np.unique(train['image_link']).shape[0])

0.969858901913522
7953


In [32]:
# save the small_train dataframe
small_train.to_csv('../small_train_1000.csv', index=False)

In [ ]:
# entity_value is of the form "Num unit"
# i want to see if the extracted text contains the number in any form. ignore the unit

def check_number_in_extracted_text(entity_value, extracted_texts):
    for text in extracted_texts:
        num = entity_value.split(' ')[0]
        if '.' in num:
            if num.split('.')[-1] in ['0', '00']:
                if num.split('.')[0] in text:
                    return True
        if num in text:
            return True
            
    return False

In [43]:
# read csv 1000_ocr_with_inference.csv
import pandas as pd
ocr_data = pd.read_csv('../dataset/1000_ocr_with_inference.csv')
ocr_data[["entity_name", "entity_value", "final_ext_val", "extracted_texts"]][40:50]

,entity_name,entity_value,final_ext_val,extracted_texts
40,item_weight,25.0 gram,0 gram,[]
41,maximum_weight_recommendation,9.0 ton,I'm not able to find that information in the g...,[]
42,depth,5.5 centimetre,5.5cm,['5.5cm (2.17in)']
43,height,11.0 inch,11 inch,"['11""', '61/8""']"
44,item_weight,3.7 kilogram,3.7kg,"['33.2cm', '15cm', '15cm', '15.8cm', '48cm', '..."
45,item_weight,100 gram,19.1g,"['tomation (Per 00g.)', '19.1g', '44.8g', 'Net..."
46,width,52.0 centimetre,53 cm,"['49-61 cm', '53 cm', '52 cm']"
47,width,28.0 centimetre,10cm,"['28cm', '10cm', '43cm', '10cm']"
48,item_weight,30.0 gram,6 pound,"['827', 'SINCE 1984', '100', '5VARIETIES:', 'S..."
49,item_weight,125.0 milligram,45 milligram,"['SIMETHICONE 125 mg / ANTIGAS', 'Simethicone ..."


In [49]:
!pip install fuzzywuzzy

In [92]:
import pandas as pd
import os

df = pd.read_csv('../dataset/train_0_to_66000_with_ocr.csv')

# convert the extracted_text column to a list
df['extracted_texts'] = df['extracted_texts'].apply(lambda x: eval(x))
df.head()

,image_link,group_id,entity_name,entity_value,extracted_texts,number_in_extracted_text
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,"[PROPOS, NATURE, INGREDIENT MENAGER, MULTI-USA...",True
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,"[TEARRIFIC, LEBENSMITTELECHT, Cup, HDAY, GEPRA...",False
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,"[COMPOSITION, Serving Size:1 Tablet 0.709 g)Ea...",True
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,"[ZRDA', :, 11180, ZRDA, Qty./Serving, EEEE, Qt...",True
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,"[Horbaach, HIGHSTRENGTH, PSYLLIOM, HUSK, 1400,...",True


In [93]:
entity_unit_map = {
  "width": {
    "centimetre" : ["centimetre", "centimeter", "cm"],
    "foot": ["foot", "ft"],
    "millimetre": ["millimetre", "millimeter", "mm"],
    "metre": ["metre", "meter", "m"],
    "inch": ["inch", "in"],
    "yard": ["yard", "yd"]
  },
  "depth": {
    "centimetre": ["centimetre", "centimeter", "cm"],
    "foot": ["foot", "ft"],
    "millimetre": ["millimetre", "millimeter", "mm"],
    "metre": ["metre", "meter", "m"],
    "inch": ["inch", "in"],
    "yard": ["yard", "yd"]
  },
  "height": {
    "centimetre": ["centimetre", "centimeter", "cm"],
    "foot": ["foot", "ft"],
    "millimetre": ["millimetre", "millimeter", "mm"],
    "metre": ["metre", "meter", "m"],
    "inch": ["inch", "in"],
    "yard": ["yard", "yd"]
  },
  "item_weight": {
    "milligram": ["milligram", "mg"],
    "kilogram": ["kilogram", "kg"],
    "microgram": ["microgram", "µg", "mcg"],
    "gram": ["gram", "g"],
    "ounce": ["ounce", "oz"],
    "ton": ["ton", "t"],
    "pound": ["pound", "lb", "lbs"]
  },
  "maximum_weight_recommendation": {
    "milligram": ["milligram", "mg"],
    "kilogram": ["kilogram", "kg"],
    "microgram": ["microgram", "µg", "mcg"],
    "gram": ["gram", "g", "gm"],
    "ounce": ["ounce", "oz"],
    "ton": ["ton", "t"],
    "pound": ["pound", "lb", "lbs"]
  },
  "voltage": {
    "millivolt": ["millivolt", "mv"],
    "kilovolt": ["kilovolt", "kv"],
    "volt": ["volt", "v"]
  },
  "wattage": {
    "kilowatt": ["kilowatt", "kw"],
    "watt": ["watt", "w"]
  },
  "item_volume": {
    "cubic foot": ["cubic foot", "cubic ft"],
    "microlitre": ["microlitre", "microliter", "µl"],
    "cup": ["cup", "cups"],
    "fluid ounce": ["fluid ounce", "fl oz", "floz"],
    "centilitre": ["centilitre", "centiliter", "cl"],
    "imperial gallon": ["imperial gallon", "gal (UK)"],
    "pint": ["pint", "pt"],
    "decilitre": ["decilitre", "deciliter", "dl"],
    "litre": ["litre", "liter", "L"],
    "millilitre": ["millilitre", "milliliter", "ml"],
    "quart": ["quart", "qt"],
    "cubic inch": ["cubic inch", "cubic in"],
    "gallon": ["gallon", "gal"]
  }
}

In [94]:
import regex as re

def extract_number(text, entity):
    text = text.lower()
    possible_matches = []
    # replace comma with .
    text = text.replace(',', '.')
    # find all number unit pairs
    for unit in entity_unit_map[entity]:
        for unit_name in entity_unit_map[entity][unit]:
            pattern = r'(\d+\.?\d*)\s*'+unit_name
            match = re.search(pattern, text)
            if match:
                possible_matches.append((match.group(1), unit))

    possible_matches = list(set(possible_matches))

    return possible_matches

def extract_matches(extracted_texts, entity):
    matches = []
    for text in extracted_texts:
        matches.extend(extract_number(text, entity))
    return list(set(matches))

In [95]:
# in the df dataframe, apply the extract_matches function to the extracted_texts column using the entity_name column
small_df = df.iloc[:1000]
small_df['matches'] = small_df.apply(lambda x: extract_matches(x['extracted_texts'], x['entity_name']), axis=1)

C:\Users\rishi\AppData\Local\Temp\ipykernel_11892\1096148830.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['matches'] = small_df.apply(lambda x: extract_matches(x['extracted_texts'], x['entity_name']), axis=1)


In [96]:
# split entity value by space and create a true num (float) and unit column
# entity value can be of type "Num unit"
# drop the rows where the entity value is not of the above type
temp_df = small_df.copy()
# add two new columns
temp_df['true_num'] = 0.0
temp_df['unit'] = ''
for i in range(small_df.shape[0]):
    entity_value = small_df['entity_value'].iloc[i]
    entity_value = entity_value.replace('[', '').replace(']', '')
    entity_value = entity_value.split(' ')
    if len(entity_value) == 2:
        temp_df.loc[i, 'true_num'] = float(entity_value[0])
        temp_df.loc[i, 'unit'] = entity_value[1]
    else:
        temp_df.drop(index=i, inplace=True)

temp_df.head(20)
y = 0
# def split_entity_value(entity_value):
#     global y
#     y += 1
#     try:
#         entity_value = entity_value.replace('[', '').replace(']', '')
#         entity_value = entity_value.split(' ')
#         if len(entity_value) == 2:
#             return [float(entity_value[0])], entity_value[1]
#         else:
#             return [float(x) for x in entity_value[:-1]], entity_value[-1]
#     except:
#         print(entity_value, y)

# # true num has list 
# small_df['true_num'] = small_df['entity_value'].apply(lambda x: split_entity_value(x)[0])
# small_df['unit'] = small_df['entity_value'].apply(lambda x: split_entity_value(x)[1])

cnt = 0
# check if any of the matches are exactly correct in temp_df
for i in range(temp_df.shape[0]):
    for match in temp_df['matches'].iloc[i]:
        if float(match[0]) == float(temp_df['true_num'].iloc[i]) and match[1] == temp_df['unit'].iloc[i]:
            cnt += 1
            break
print(cnt)
temp_df[['matches', 'true_num', 'unit']].head(20)

685


,matches,true_num,unit
0,"[(500, gram)]",500.000,gram
1,[],1.000,cup
2,"[(0.51, gram), (25, milligram), (5, milligram)...",0.709,gram
3,"[(0.709, gram), (1, ton)]",0.709,gram
4,[],1400.000,milligram
5,"[(1400, milligram)]",1400.000,milligram
6,"[(1400, milligram)]",1400.000,milligram
7,"[(1400, milligram)]",1400.000,milligram
8,"[(1400, milligram)]",1400.000,milligram
9,"[(1400, milligram)]",1400.000,milligram


In [102]:
import numpy as np
np.random.seed(0)

def handle_no_matches(entity_name, extracted_texts):
    # search for any float or int number in the extracted texts using regex
    poss_units = []
    for unit in entity_unit_map[entity_name]:
        poss_units.append(unit)
        
    if len(poss_units) > 6:
        return ""

    for text in extracted_texts:
        match = re.search(r'(\d+\.?\d*)', text)
        if match:
            # round to 1 decimal place
            num = float(match.group(1))
            num = '{:.1f}'.format(round(num, 1))
            idx = np.random.choice(len(poss_units))
            return str(num) + ' ' + poss_units[idx]
    
    return ""

def predict_entity(matches, entity_name, extracted_texts):
    if len(matches) == 0:
        return handle_no_matches(entity_name, extracted_texts)
    
    if len(matches) == 1:
        num = float(matches[0][0])
        num = '{:.1f}'.format(round(num, 1))
        return str(num) + ' ' + matches[0][1]
    
    if len(matches) > 1:
        if len(matches) > 3:
            return ""
        if entity_name == "width":
            mx = 0
            for match in matches:
                if float(match[0]) > mx:
                    mx = float(match[0])
            mx = '{:.1f}'.format(round(mx, 1))
            return str(mx) + ' ' + matches[0][1]
        
        idx = np.random.choice(len(matches))
        num = float(matches[idx][0])
        num = '{:.1f}'.format(round(num, 1))

        return str(num) + ' ' + matches[idx][1]
    
temp_df['predicted_entity_value'] = temp_df.apply(lambda x: predict_entity(x['matches'], x['entity_name'], x['extracted_texts']), axis=1)
temp_df[['entity_value', 'predicted_entity_value']].head(20)

,entity_value,predicted_entity_value
0,500.0 gram,500 gram
1,1.0 cup,
2,0.709 gram,
3,0.709 gram,0.709 gram
4,1400 milligram,
5,1400 milligram,1400 milligram
6,1400 milligram,1400 milligram
7,1400 milligram,1400 milligram
8,1400 milligram,1400 milligram
9,1400 milligram,1400 milligram


In [98]:
# calculate accuracy
cnt = 0
for i in range(temp_df.shape[0]):
    # check float values
    if float(temp_df['predicted_entity_value'].iloc[i].split(' ')[0]) == float(temp_df['true_num'].iloc[i]):
        if temp_df['predicted_entity_value'].iloc[i].split(' ')[1] == temp_df['unit'].iloc[i]:
            cnt += 1

accuracy = cnt/temp_df.shape[0]
print(accuracy, cnt, temp_df.shape[0])

0.5711297071129707 546 956


In [107]:
# a true positive is when float value and unit are both correct
# a false positive is when ground truth is empty and predicted value is not empty
# a false negative is when ground truth is not empty and predicted value is empty
# a true negative is when both ground truth and predicted value are empty

true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0

for i in range(temp_df.shape[0]):
    if temp_df['predicted_entity_value'].iloc[i] == '':
        if temp_df['entity_value'].iloc[i] == '':
            true_negatives += 1
        else:
            false_negatives += 1
    else:
        if temp_df['entity_value'].iloc[i] == '':
            false_positives += 1
        else:
            if float(temp_df['predicted_entity_value'].iloc[i].split(' ')[0]) == float(temp_df['true_num'].iloc[i]):
                if temp_df['predicted_entity_value'].iloc[i].split(' ')[1] == temp_df['unit'].iloc[i]:
                    true_positives += 1
                else :
                    false_positives += 1
            else:
                false_positives += 1


In [108]:
precision = true_positives/(true_positives + false_positives)
recall = true_positives/(true_positives + false_negatives)
f1_score = 2*precision*recall/(precision + recall)

print(precision, recall, f1_score)
accuracy = (true_positives + true_negatives)/(true_positives + true_negatives + false_positives + false_negatives)
print(accuracy)
print(true_positives, false_positives, false_negatives, true_negatives)

0.7684797768479776 0.6974683544303798 0.7312541473125413
0.5763598326359832
551 166 239 0


In [109]:
false_negatives = 58000
true_negatives = 0
positives = 131187 - false_negatives
f1 = 0.39

# find the number of false positives using this information
false_positives = 0
precision = positives/(positives + false_positives)

57960.0

In [120]:
idx = 10
temp_df[temp_df['predicted_entity_value'] == ''].head(idx)[["extracted_texts", "entity_name", "entity_value", "matches"]]

,extracted_texts,entity_name,entity_value,matches
1,"[TEARRIFIC, LEBENSMITTELECHT, Cup, HDAY, GEPRA...",item_volume,1.0 cup,[]
2,"[COMPOSITION, Serving Size:1 Tablet 0.709 g)Ea...",item_weight,0.709 gram,"[(0.51, gram), (25, milligram), (5, milligram)..."
4,"[Horbaach, HIGHSTRENGTH, PSYLLIOM, HUSK, 1400,...",item_weight,1400 milligram,[]
15,"[XX X X, L.oose packaging, Impurities added, O...",item_weight,100 gram,[]
18,"[FREE, Glucon-D, Regular 200gpack, Glucon-D, I...",item_weight,200 gram,"[(170, milligram), (59.5, milligram), (35, mil..."
20,"[HEF, Glucon-, 9.4in, 23cm]",item_weight,200 gram,[]
21,"[ONE SIZE, FItS, 6 ALL, STRETCHABLE AND, BLACK...",item_volume,4.0 gallon,[]
34,"[65:62, USB, CHARGING, PORTS, 50, MODE, UP, D...",item_weight,50.0 gram,[]
39,"[51*70 IN, 31 0Z, GARDEN, soft, ventilate, wa...",item_weight,31.0 ounce,[]
41,"[hot'chocolate, BOMBS, Made with Belgian Choco...",item_weight,35.0 gram,"[(0.6, milligram), (5, milligram), (0, gram), ..."


In [119]:
(temp_df[temp_df['predicted_entity_value'] == ''].head(idx)["image_link"].tolist())

['https://m.media-amazon.com/images/I/71gSRbyXmoL.jpg',
 'https://m.media-amazon.com/images/I/61BZ4zrjZXL.jpg',
 'https://m.media-amazon.com/images/I/617Tl40LOXL.jpg',
 'https://m.media-amazon.com/images/I/91-iahVGEDL.jpg',
 'https://m.media-amazon.com/images/I/81RNsNEM1EL.jpg',
 'https://m.media-amazon.com/images/I/31EvJszFVfL.jpg',
 'https://m.media-amazon.com/images/I/61wzlucTREL.jpg',
 'https://m.media-amazon.com/images/I/71004c9tzfL.jpg',
 'https://m.media-amazon.com/images/I/915JHkwtcrL.jpg',
 'https://m.media-amazon.com/images/I/81hnk2WXO3L.jpg']

In [128]:

# predicted entity value is empty and number in extracted text is True
temp_df[(temp_df['predicted_entity_value'] == '') & (temp_df['number_in_extracted_text'] == True)].iloc[2].to_dict()

{'image_link': 'https://m.media-amazon.com/images/I/81RNsNEM1EL.jpg',
 'group_id': 731432,
 'entity_name': 'item_weight',
 'entity_value': '200 gram',
 'extracted_texts': ['FREE',
  'Glucon-D',
  'Regular 200gpack',
  'Glucon-D',
  'Instant Energy',
  'NUTRITIONALVALUE*',
  'Per 100g Per Serve (35g)',
  'Energy Value.',
  '360kcal',
  '126kcal',
  'Prot.e.',
  'Og',
  'Carbohydrates.',
  '31.5g',
  'of which sugar (sucrose)..',
  '.',
  'Og',
  'Fat&All types of ftty acid.',
  'Og',
  'Calc.iu.',
  '.170mg',
  '59.5mg',
  'Phosphorus.',
  '.100mg',
  '35mg',
  'Vitamin D.',
  '.3001.U.',
  '1051.U.',
  '..',
  '*Approx.Value',
  'R5s0i',
  'NGREDIENTSGlcose 99.4%)Mirals CamPhsphates andVitain.',
  'Glucose based beverage mix Proprietary Food).',
  'Storage ConditionStore in a cooldry place away from strong odours.',
  'BEST BEFORE 24 MONTHS FROM MANUFACTURE.',
  'Heinz',
  'Manufactured forHeinz ndiaPt.td.7FlorDShivsagar',
  'Worli, Mumbai-400 018.',
  'fssai',
  'Lic.No.10013022001417

In [129]:
# drop the rows where true_num is -1
temp_df = temp_df[temp_df['true_num'] != -1]

,image_link,group_id,entity_name,entity_value,extracted_texts,number_in_extracted_text,matches,true_num,unit,predicted_entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,"[PROPOS, NATURE, INGREDIENT MENAGER, MULTI-USA...",True,"[(500, gram)]",500.0,gram,500 gram
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,731432,item_weight,1400 milligram,"[Horbaach, HIGHSTRENGTH, Naturally-Sourced Psy...",True,"[(1400, milligram)]",1400.0,milligram,1400 milligram
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,731432,item_weight,1400 milligram,"[Horbaach, Directions: For adults, take two (2...",True,"[(1400, milligram)]",1400.0,milligram,1400 milligram
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,731432,item_weight,1400 milligram,"[VEGAN, Horbaach, WHEAT, FREE, HIGHSTRENGTH, 0...",True,"[(1400, milligram)]",1400.0,milligram,1400 milligram
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,731432,item_weight,1400 milligram,"[Horbaach, 100%, HIGHEST, QUALITY, Horbaach, T...",True,"[(1400, milligram)]",1400.0,milligram,1400 milligram
...,...,...,...,...,...,...,...,...,...,...
991,https://m.media-amazon.com/images/I/61Aky4kqZj...,641642,voltage,220.0 volt,"[LED CONTROLLER, CE, 220V3CH, OUUTPT]",True,"[(220, volt)]",220.0,volt,220 volt
992,https://m.media-amazon.com/images/I/61HF-lj-xx...,589105,item_weight,125.0 gram,"[Weight:125g, 21cm(8.3in), 12.5cm(5in), 5.5cm(...",True,"[(125, gram)]",125.0,gram,125 gram
994,https://m.media-amazon.com/images/I/61ZCwCtSyY...,969033,maximum_weight_recommendation,10 kilogram,"[35cm/14in, 50cm/19in, up to 10kg]",True,"[(10, kilogram)]",10.0,kilogram,10 kilogram
998,https://m.media-amazon.com/images/I/81fYO+HP5Q...,186035,voltage,0.5 volt,"[SHUTOF, APASADQAUTOMATICO, 12VDC 15Ah/20Hr]",False,"[(12, volt)]",0.5,volt,12 volt
